In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from skimage.color import rgb2gray
from skimage.io import imread, imsave, imshow
from tqdm.auto import tqdm

# Increase these if figures appear small
plt.rcParams["figure.figsize"] = fx, fy = (19.20, 5.40)

### Познакомимся с данными

Загрузим и посмотрим на изображение

In [ ]:
!curl -O 'https://code.mipt.ru/courses-public/cv/storage/-/raw/seminars/basic-image-processing/msu.jpg'

In [ ]:
img = imread("msu.jpg")

In [ ]:
[img.dtype, img.shape]

In [ ]:
[img.min(), img.mean(), img.max()]

In [ ]:
plt.imshow(img)
plt.show()

Что-то небо слишком синее. Давайте попробуем сделать его потемнее.

In [ ]:
img_dark_sky = img.copy()
...

In [ ]:
plt.imshow(img_dark_sky)
plt.show()

Ой! Деревья стали синими! Что пошло не так?

---

Давайте лучше будем работать в `float32` в диапазоне $\left[0.0;\;1.0\right]$

In [ ]:
img_f32 = ...

In [ ]:
[img_f32.dtype, img_f32.shape]

In [ ]:
[img_f32.min(), img_f32.mean(), img_f32.max()]

In [ ]:
plt.imshow(img_f32)
plt.show()

А теперь?

In [ ]:
img_dark_sky_f32 = img_f32.copy()
...

In [ ]:
plt.imshow(img_dark_sky_f32)
plt.show()

---

Сконвертируем изображение в черно-белое

In [ ]:
gray_f32 = rgb2gray(img_f32)

In [ ]:
[gray_f32.dtype, gray_f32.shape]

In [ ]:
plt.imshow(gray_f32)
plt.show()

Ой! Что пошло не так?

Давайте попробуем сохранить картинку и посмотреть на неё без `matplotlib`.

In [ ]:
msu_gray_u8 = ...

imsave("msu_gray.png", msu_gray_u8)

Откроем сохраненный файл [`msu_gray.png`](./msu_gray.png).

---

Исправим проблему

In [ ]:
plt.imshow(gray_f32, cmap="gray")
plt.show()

Теперь все нормально?

---

Давайте сконвертируем в черно-белое изображение сами

In [ ]:
simple_manual_gray_f32 = ...

In [ ]:
[simple_manual_gray_f32.dtype, simple_manual_gray_f32.shape]

И сравним с библиотечным

In [ ]:
plt.subplot(1, 2, 1)
plt.imshow(gray_f32, cmap="gray")
plt.title("gray_f32")

plt.subplot(1, 2, 2)
plt.imshow(simple_manual_gray_f32, cmap="gray")
plt.title("simple_manual_gray_f32")

plt.show()

In [ ]:
plt.imshow(img)
plt.show()

Трава на нашей версии почему-то чуть темнее

In [ ]:
abs(simple_manual_gray_f32 - gray_f32).max()

Хм. Давайте посмотрим, что говорят в документации библиотечной версии.

In [ ]:
help(rgb2gray)

In [ ]:
...
crt_luminance_f32 = ...

In [ ]:
[crt_luminance_f32.dtype, crt_luminance_f32.shape]

In [ ]:
plt.subplot(1, 2, 1)
plt.imshow(gray_f32, cmap="gray")
plt.title("gray_f32")

plt.subplot(1, 2, 2)
plt.imshow(crt_luminance_f32, cmap="gray")
plt.title("crt_luminance_f32")

plt.show()

In [ ]:
plt.imshow(img_f32)
plt.title("img_f32")
plt.show()

In [ ]:
abs(crt_luminance_f32 - gray_f32).max()

Вот, другое дело

### Работа с гистограммой изображения

Гистограмма изображения позволяет оценить количество и разнообразие оттенков, а также общий уровень яркости. Гистограмма изображения не описывает художественных качеств, но с её помощью можно рассчитывать разные статистики в картинке. Например найти миниум, максимум, среднее, медиану.

In [ ]:
def plot_image_with_hist(image):
    # flatten image cause in image's histogram
    # there is no need to keep geometric information
    values = ...

    plt.subplot(1, 2, 1)
    # plot the histogram
    ...

    plt.subplot(1, 2, 2)
    # show the image
    plt.imshow(image, cmap="gray")


plot_image_with_hist(gray_f32)

Самым простым видом трансформаций цветовых компотент изображений являются точечные операторы, которые изменяют каждый пиксель независимо. Например понизить яркость картинки можно поделив каждый пиксель на константу.

$$ value = value\ /\ 3 $$

Реализуем это преобразование и посмотрим на результат

In [ ]:
darkened_f32 = gray_f32 / 3

In [ ]:
plot_image_with_hist(darkened_f32)

In [ ]:
darkened_u8 = ...
darkened_fp32 = ...
brigthened_fp32 = ...

In [ ]:
plot_image_with_hist(brigthened_fp32)

In [ ]:
plot_image_with_hist(3 * darkened_f32)

Так. Что-то опять не получается.

Давайте исправим диапазон гистограммы и изображения.

---

#### Гамма-коррекция

Можно выделить общую форму для всех операторов преобразования изображений, как функцию от одной или более картинок, которая отображает аргументы в итоговую картинку.

$$ y = g(x_{0}, x_{1}, ..., x_{n}) $$

Так например, гамма коррекция задается формулой

$$ y = g(x) = x^{\gamma} $$

Во времена черно-белых телевизоров фосфор в электронно-лучевых трубках нелинейно реагировал на входное напряжение. Отношение между входным напряжением и получающейся яркостью можно было описать гаммой $ \gamma $, поскольку с грубой оценкой выполнялось $$ brightness = voltage^{\gamma}. $$ Где $ \gamma = 2.2 $.
Чтобы обратить этот эффект телевизоры перед подачей кадра обрабатывали его через $$ Y' = Y^{\frac{1}{\gamma}}, $$
$$ \dfrac{1}{\gamma} = 0.45. $$


In [ ]:
gamma = 1 / 2.2

gamma_correction = ...

plot_image_with_hist(gamma_correction)

#### Линейное контрастирование

$$ y = g(x) = \frac{x - x_{min}}{x_{max} - x_{min}} $$

Используется чтобы повысить контрастность изображения. Один из самых примитивных и не очень хорошо работающих методов.

In [ ]:
lin_stretch_img = ...

plot_image_with_hist(lin_stretch_img)

Как ещё мы можем повысить контрастность изображения? Чтобы сгладить резкие углы, используется эквивализация гистограммы. Алгоритм для выравнивания гистограммы цветов.

### Histogram equalization

Как найти такое преобразование? Трюк тот же, что и при сэмплировании случайных чисел для разнообразных распределений с использованием функции распределения $F(x)$. Ответом является отображение значений пикселей в соответствующие им квантили.

Почему?
> После такого преобразования кумилятивная функция распределения итоговых значений $F(x)$ в идеальной картине мира без квантизации будет $F(x) = x$.
>
> Следовательно плотность распределения пикселов (гистограмма), будет равномерной.

Алгоритм:
1. Посчитать гистограмму цветов h
2. Посчитать префиксные суммы массива h, оно и есть CDF(cumulative distribution function) цветов картинки
3. Чтобы получить картинку с линеаризованной CDF используем трансформацию пикселов по правилу: $ y = CDF(x) $, чтобы получить квантиль для исходного значения пикселя x.

Подробнее об алгоритме можно почитать на wiki [Histogram_equalization](https://en.wikipedia.org/wiki/Histogram_equalization#Implementation)

In [ ]:
def calculate_cdf(gray_image):
    hist, _ = ...
    cdf = ...
    return cdf


def hist_equalization(gray_image):
    cdf = calculate_cdf(gray_image)

    # map all values to cdf's values
    gray_image = ...

    return gray_image

In [ ]:
!curl -O 'https://code.mipt.ru/courses-public/cv/storage/-/raw/seminars/basic-image-processing/tests.py'

In [ ]:
# Тесты
from tests import test_he

test_he(hist_equalization)

In [ ]:
he_f32 = hist_equalization(gray_f32)

plot_image_with_hist(he_f32)

In [ ]:
def plot_compare(images, names):
    for idx, (img, title) in enumerate(zip(images, names, strict=True)):
        plt.subplot(1, len(images), idx + 1)
        if img.ndim != 3:
            img = np.repeat(img[..., None], 3, axis=-1)
        plt.imshow(img)
        plt.title(title)
        plt.axis("off")

In [ ]:
images = [gray_f32, lin_stretch_img, gamma_correction, he_f32]
names = ["original", "linear stretching", "gamma correction", "histogram equalization"]

plot_compare(images, names)

### Locally adaptive histogram equalization

Глобальная гистограмма цветов может быть очень полезной, но в то же время для некоторых картинок предпочтительней применить различного вида эквивализации к разным регионам. Добиться этого можно с помощью разбиения картинки на блоки размера $(M, M)$ и применить $HE$ независимо для каждого блока. Если блоки будут не пересекаться, то в картинке могут появиться артефакты.

In [ ]:
def naive_lahe_filt(image, block_size=32):
    # for simplicity, lets crop the image to a multiple of the block size
    ylen = image.shape[0] // block_size
    xlen = image.shape[1] // block_size
    image = image[: ylen * block_size, : xlen * block_size]

    new_image = np.zeros_like(image)

    for i in range(ylen):
        y = block_size * i
        for j in range(xlen):
            x = block_size * j

            # Extract the block
            block = ...

            # Equalize histogram for this block
            block = ...

            # Write the block back
            new_image[...] = block

    return new_image

In [ ]:
naive_lahe_f32 = naive_lahe_filt(gray_f32)

plot_image_with_hist(naive_lahe_f32)

Чтобы избежать таких артефактов, можно использовать движущееся окно, а именно рассчитывать гистограммы для каждого блока $(M, M)$ и менять значение только в его центре.

Давайте реализуем алгоритм через наш обобщенный алгоритм фильтрации.

### Фильтрация изображений

![](https://code.mipt.ru/courses-public/cv/storage/-/raw/seminars/basic-image-processing/filt.png)

Фильтрацией изображений называют преобразование пикселей изображений с учетом региона определенной формы.

$$ g(x)_{ij} = k\left(x_{\mathrm{region}\left(i,\, j\right)}\right)$$

Функция $k$ применяется к каждому региону изображения и называется ядром фильтра.

Давайте реализуем обобщенный алгоритм для фильтрации изображений

In [ ]:
def filt(image, kernel, *, kernel_size=3, padd=0, fill_mode="reflect"):
    """
    kernel - arbitrary function with one argument - window
    fill_mode - 'constant', 'symmetric', 'reflect'
    """
    if padd:
        padding = [(padd, padd), (padd, padd)]
        if image.ndim == 3:
            padding += [(0, 0)]

        # use np.pad for image padding
        image = ...

    h_range = range(image.shape[0] - (kernel_size - 1))
    w_range = range(image.shape[1] - (kernel_size - 1))
    channels = image.shape[-1:] if image.ndim == 3 else ()

    # define new image
    new_image = ...

    for i in tqdm(h_range, total=len(h_range)):
        for j in w_range:
            # define image slice - window
            window = ...

            # apply kernel to window
            value = ...

            # write out the result
            new_image[...] = value

    return new_image

In [ ]:
# Тесты
from tests import test_filt

test_filt(filt)

![](https://code.mipt.ru/courses-public/cv/storage/-/raw/seminars/basic-image-processing/ahe3.png)

In [ ]:
def lahe_kernel(window):
    h, w = window.shape[:2]

    # calculate cdf of image's color histogram
    cdf = calculate_cdf(window)

    # extract the middle pixel from the window
    center = ...

    # map only the extracted value using the whole cdf
    value = ...

    return value

In [ ]:
# Тесты
from tests import test_lahe

test_lahe(filt, lahe_kernel)

In [ ]:
lahe_f32 = filt(gray_f32, lahe_kernel, kernel_size=51, padd=25, fill_mode="reflect")
plot_image_with_hist(lahe_f32)

### Шум

Цифровой шум - это дефект изображения, вносимый фотосенсорами и электроникой устройств(как черно-белые телевизоры портили нам яркость) из-за несовершенства технологий или фотонной природы света. Некоторые виды шумов можно промоделировать и использовать для контроля качества алгоритмов понижения уровня шума на картинке. Например, гауссовским шумом на изображении называется колебание яркости распределенное по нормальному закону.

### Drop-out noise

Или по-другому потеря информации. Соль и перец - случайные черные и белые пиксели, а только белые - импульсивный.

In [ ]:
def sample_idx(h, w, count=500):
    y = np.random.randint(h, size=count)
    x = np.random.randint(w, size=count)
    return y, x


def dropout_values(image, val, count=500):
    image = image.copy()
    random_coords = sample_idx(*image.shape[:2], count=count)
    image[random_coords] = val
    return image


def salt_pepper(image, count=5000):
    # random white and black pixels
    image = dropout_values(image, 1, count=count)
    image = dropout_values(image, 0, count=count)
    return image


# only white pixels
impulsive_f32 = dropout_values(img_f32, 1, count=5000)
# plus dark
salt_peper_f32 = salt_pepper(img_f32)

images = [img_f32, impulsive_f32, salt_peper_f32]
names = ["original", "impulsive noise", "salt and peper noise"]

plot_compare(images, names)

### Median filter

Медианный фильт - один из нелинейных фильтров, используемых для уменьшения уровня шума в изображении. Результатом преобразования одного региона является медиана его чисел.

In [ ]:
def median_filter(window):
    channels = window.shape[-1:] if window.ndim == 3 else ()

    # sort values
    values = ...

    # choose mid value
    median = ...

    return median

In [ ]:
# Тесты
from tests import test_median

test_median(filt, median_filter)

In [ ]:
for name, noise in [
    ("impulsive noise", impulsive_f32),
    ("salt and peper noise", salt_peper_f32),
]:
    images = [img_f32, noise, filt(noise, median_filter, kernel_size=3, padd=1)]
    names = ["original", name, f"{name} + median filter"]

    plot_compare(images, names)
    plt.show()

In [ ]:
noisy_f32 = (img_f32 + np.random.randn(*img_f32.shape) * 0.1).clip(0, 1)
plt.imshow(noisy_f32)
plt.show()

In [ ]:
images = [img_f32, noisy_f32, filt(noisy_f32, median_filter, kernel_size=3, padd=1)]
names = ["original", "noisy", "noisy+median filter"]


plot_compare(images, names)

### Границы объектов

Основной целью обнаружения резких изменений яркости изображения является фиксация важных событий и изменений мира. Они могут отражать различные предположения о модели формирования изображения, изменения в яркости изображения могут указывать на:

> изменения глубины;

> изменения ориентации поверхностей;

> изменения в свойствах материала;

> различие в освещении сцены.

В идеальном случае результатом выделения границ является набор связанных кривых, обозначающих границы объектов. Таким образом, применение фильтра выделения границ к изображению может существенно уменьшить количество обрабатываемых данных из-за того, что отфильтрованная часть изображения считается менее значимой, а наиболее важные структурные свойства изображения сохраняются.

Один из самых простых методов выделения границ - оператор Собеля.

![](https://code.mipt.ru/courses-public/cv/storage/-/raw/seminars/basic-image-processing/sobel.png)


![](https://code.mipt.ru/courses-public/cv/storage/-/raw/seminars/basic-image-processing/grad_norm.png)

In [ ]:
sobel_kernel = np.array(
    [
        [1.0, 2.0, 1.0],
        [0.0, 0.0, 0.0],
        [-1.0, -2.0, -1.0],
    ]
)


def sobel_x(window):
    return ...


def sobel_y(window):
    return ...

In [ ]:
def sobel(gray_image):
    der_x = filt(gray_image, sobel_x, kernel_size=3, padd=1, fill_mode="reflect")
    der_y = filt(gray_image, sobel_y, kernel_size=3, padd=1, fill_mode="reflect")

    edge = ...
    return edge

In [ ]:
# Тесты
from tests import test_sobel

test_sobel(sobel)

In [ ]:
edge = sobel(gray_f32)

plt.imshow(edge)
plt.colorbar()
plt.show()

### Max filter

In [ ]:
def max_filter(window):
    # return max-value among height and width
    return ...

In [ ]:
# Тесты
from tests import test_max

test_max(filt, max_filter)

In [ ]:
noise_f32 = (img_f32 + 0.5 * np.random.randn(*img_f32.shape)).clip(0, 1)
median_f32 = filt(noise_f32, median_filter, kernel_size=5, padd=2)
max_f32 = filt(median_f32, max_filter, kernel_size=5, padd=2)

res = (edge[:, :, None] * max_f32).clip(0, 1)

plt.imshow(res)
plt.show()